<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/FourSqure_CV%E7%A2%BA%E8%AA%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for colab

In [2]:
import sys
is_colab = False
if "google.colab" in sys.modules:
  is_colab = True

In [3]:
if is_colab:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
if is_colab:
  !mkdir -p ~/.kaggle
  !cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/
  !chmod 600 /root/.kaggle/kaggle.json

In [5]:
if is_colab:
  !pip install kaggle
  !apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [6]:
if is_colab:
  !mkdir input output

In [7]:
if is_colab:
  !kaggle competitions download -c foursquare-location-matching
  !unzip foursquare-location-matching.zip -d input/foursquare-location-matching

 95% 150M/159M [00:00<00:00, 224MB/s]
100% 159M/159M [00:00<00:00, 227MB/s]
Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [8]:
if is_colab:
  !kaggle datasets download -d guoyonfan/binary-lgb-baseline
  !unzip /content/binary-lgb-baseline.zip -d input/binary-lgb-baseline

  0% 0.00/3.90M [00:00<?, ?B/s]
100% 3.90M/3.90M [00:00<00:00, 133MB/s]
Archive:  /content/binary-lgb-baseline.zip
  inflating: input/binary-lgb-baseline/improved_lgb_baseline.lgb  
  inflating: input/binary-lgb-baseline/lgb_baseline.lgb  


In [9]:
if is_colab:
  !cp input/binary-lgb-baseline/*  /content/drive/MyDrive/datas/foursquare

In [10]:
if is_colab:
  !pip install Levenshtein
  !pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 258 kB 29.0 MB/s 
     |████████████████████████████████| 1.8 MB 45.1 MB/s 
     |████████████████████████████████| 103 kB 45.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 74 kB/s 


# change directory

In [11]:
if is_colab:
  %cd /content/output

/content/output


In [12]:
if is_colab:
    !pip install pykakasi
    !pip install fuzzywuzzy
else :
    !unzip ../input/foursquare-lightgbm-trained/pykakasi_deps.dontopenthiskaggle/pykakasi_deps.dontopenthiskaggle -d .
    !conda install ./pykakasi_deps/offline_pykakasi.tar.bz2
    !conda install ./pykakasi_deps/offline_jaconv.tar.bz2
    !conda install ./pykakasi_deps/offline_deprecated.tar.bz2
    !pip install ../input/foursquare-lightgbm-trained/fuzzywuzzy-0.18.0-py2.py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 27.6 MB/s 
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15564 sha256=15c1fb608856de91feeac50d63ff8525184d15104c3f764368ae6a3df81f384b
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### This notebook is the improved version of [binary_lgb_baseline](https://www.kaggle.com/code/guoyonfan/binary-lgb-baseline-0-834), and mainly benefits from the [simple_recall_method](https://www.kaggle.com/code/guoyonfan/simple-recall-method). This notebook improves the LB score of the binary_lgb_baseline from 0.834 to 0.839.

### If you find these notebooks useful, please UPVOTE!

In [13]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import Levenshtein
import difflib
import multiprocessing
import pandas as pd
import numpy as np
import lightgbm as lgb
from collections import Counter
from tqdm.auto import tqdm
from tqdm.notebook import tqdm

tqdm.pandas()

from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from catboost import CatBoostClassifier
import pykakasi

from fuzzywuzzy import fuzz

In [14]:
TRAIN_FEATURES = features = ['kdist', 'kneighbors', 'kdist_country',
       'kneighbors_country',  'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'name_len_diff', 'name_nleven',
       'name_nlcsk', 'name_nlcs', 'address_sim', 'address_gesh',
       'address_leven', 'address_jaro', 'address_lcs', 'address_fuz',
       'address_len_diff', 'address_nleven', 'address_nlcsk', 'address_nlcs',
       'city_gesh', 'city_leven', 'city_jaro', 'city_lcs', 'city_fuz',
       'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs', 'state_sim',
       'state_gesh', 'state_leven', 'state_jaro', 'state_lcs', 'state_fuz',
       'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
       'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'zip_fuz', 'url_sim',
       'url_gesh', 'url_leven', 'url_jaro', 'url_lcs', 'url_fuz',
       'url_len_diff', 'url_nleven', 'url_nlcsk', 'url_nlcs', 'phone_gesh',
       'phone_leven', 'phone_jaro', 'phone_lcs', 'phone_fuz', 'categories_sim',
       'categories_gesh', 'categories_leven', 'categories_jaro',
       'categories_lcs', 'categories_fuz', 'categories_len_diff',
       'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
       'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
       'country_lcs', 'country_fuz', 'country_len_diff', 'country_nleven',
       'country_nlcsk', 'country_nlcs', 'kdist_diff',
       'kneighbors_mean']

In [57]:
## Parameters
NUM_NEIGHBOR = 3
NUM_NEIGHBOR2 = 40
SEED = 2022
THRESHOLD = 0.5
NUM_SPLIT = 10
feat_columns = ['dist', 'name', 'address', 'city', 
            'state', 'zip', 'url', 
           'phone', 'categories', 'country']
vec_columns = ['name', 'categories', 'address', 
               'state', 'url', 'country']
rec_columns = ['name', 'address', 'categories', 'address', 'phone']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(SEED)

In [16]:
%load_ext Cython

In [17]:
%%cython
def LCS(str S, str T):
    cdef int i, j
    cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
    for i in range(len(S)):
        for j in range(len(T)):
            dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
    return dp[len(S)][len(T)]

In [18]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue

        for m in match:
            if base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 

# get manhattan distance
def manhattan(lat1, long1, lat2, long2):
    return np.abs(lat2 - lat1) + np.abs(long2 - long1)

# get haversine distance
def vectorized_haversine(lats1, lats2, longs1, longs2):
    radius = 6371
    dlat=np.radians(lats2 - lats1)
    dlon=np.radians(longs2 - longs1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lats1)) \
        * np.cos(np.radians(lats2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c
    return d

In [19]:
def recall_simple(df):
    threshold = 2
    
    val2id_d = {}
    for col in rec_columns:
        temp_df = df[['id', col]]
        temp_df[col] = temp_df[col].str.lower()
        val2id = temp_df.groupby(col)['id'].apply(set).to_dict()
        val2id_d[col] = val2id
        del val2id
    
    cus_ids = []
    match_ids = []
    for vals in tqdm(df[rec_columns + ['id']].fillna('null').values):
        cus_id = vals[-1]
        match_id = []
        
        rec_match_count = []
        for i in range(len(rec_columns)):
            col = rec_columns[i]
            
            if vals[i] != 'null':
                rec_match_count += list(val2id_d[col][vals[i].lower()])
        rec_match_count = dict(Counter(rec_match_count))
        
        for k, v in rec_match_count.items():
            if v > threshold:
                match_id.append(k)
        
        cus_ids += [cus_id] * len(match_id)
        match_ids += match_id
    
    train_df = pd.DataFrame()
    train_df['id'] = cus_ids
    train_df['match_id'] = match_ids
    train_df = train_df.drop_duplicates()
    del cus_ids, match_ids
    
    num_data = len(train_df)
    num_data_per_id = num_data / train_df['id'].nunique()
    print('Num of data: %s' % num_data)
    print('Num of data per id: %s' % num_data_per_id)
    
    return train_df

In [20]:
def recall_knn(df, Neighbors = 10, Neighbors2 = 10):
    print('Start knn grouped by country')
    train_df_country = []
    for country, country_df in tqdm(df.groupby('country')):
        country_df = country_df.reset_index(drop = True)

        neighbors = min(len(country_df), Neighbors)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(country_df[['latitude','longitude']], country_df.index)
        dists, nears = knn.kneighbors(country_df[['latitude', 'longitude']], 
                                        return_distance = True)

        for k in range(neighbors):            
            cur_df = country_df[['id']]
            cur_df['match_id'] = country_df['id'].values[nears[:, k]]
            cur_df['kdist_country'] = dists[:, k]
            cur_df['kneighbors_country'] = k
            
            train_df_country.append(cur_df)
    train_df_country = pd.concat(train_df_country)
    
    print('Start knn')
    train_df = []
    knn = NearestNeighbors(n_neighbors = Neighbors2)
    knn.fit(df[['latitude','longitude']], df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']])
    
    for k in range(Neighbors2):            
        cur_df = df[['id']]
        cur_df['match_id'] = df['id'].values[nears[:, k]]
        cur_df['kdist'] = dists[:, k]
        cur_df['kneighbors'] = k
        train_df.append(cur_df)
    
    train_df = pd.concat(train_df)
    train_df = train_df.merge(train_df_country,
                                 on = ['id', 'match_id'],
                                 how = 'outer')
    del train_df_country
    
    return train_df

In [21]:
def add_features(df):    
    for col in tqdm(feat_columns):
        if col == 'dist':
            lat = data.loc[df['id']]['latitude'].values
            match_lat = data.loc[df['match_id']]['latitude'].values
            lon = data.loc[df['id']]['longitude'].values
            match_lon = data.loc[df['match_id']]['longitude'].values
            df['latdiff'] = (lat - match_lat)
            df['londiff'] = (lon - match_lon)
            df['manhattan'] = manhattan(lat, lon, match_lat, match_lon)
            df['euclidean'] = (df['latdiff'] ** 2 + df['londiff'] ** 2) ** 0.5
            df['haversine'] = vectorized_haversine(lat, match_lat, lon, match_lon)
            continue
        
        col_values = data.loc[df['id']][col].values.astype(str)
        matcol_values = data.loc[df['match_id']][col].values.astype(str)
        
        if col in vec_columns:
            tv_fit = tfidf_d[col]
            indexs = [id2index_d[i] for i in df['id']]
            match_indexs = [id2index_d[i] for i in df['match_id']]                    
            df[f'{col}_sim'] = tv_fit[indexs].multiply(tv_fit[match_indexs]).\
                                            sum(axis = 1).A.ravel()
        
        geshs = []
        levens = []
        jaros = []
        lcss = []
        fuzwuz = []
        for s, match_s in zip(col_values, matcol_values):
            if s != 'nan' and match_s != 'nan':                    
                geshs.append(difflib.SequenceMatcher(None, s, match_s).ratio())
                levens.append(Levenshtein.distance(s, match_s))
                jaros.append(Levenshtein.jaro_winkler(s, match_s))
                lcss.append(LCS(str(s), str(match_s)))
                fuzwuz.append(fuzz.ratio(s, match_s))
            else:
                geshs.append(np.nan)
                levens.append(np.nan)
                jaros.append(np.nan)
                lcss.append(np.nan)
                fuzwuz.append(np.nan)
        
        df[f'{col}_gesh'] = geshs
        df[f'{col}_leven'] = levens
        df[f'{col}_jaro'] = jaros
        df[f'{col}_lcs'] = lcss
        df[f'{col}_fuz'] = fuzwuz
        
        if col not in ['phone', 'zip']:
            df[f'{col}_len'] = list(map(len, col_values))
            df[f'match_{col}_len'] = list(map(len, matcol_values)) 
            df[f'{col}_len_diff'] = np.abs(df[f'{col}_len'] - df[f'match_{col}_len']) /\
                                        df[f'{col}_len'] 
            df[f'{col}_nleven'] = df[f'{col}_leven'] / \
                                    df[[f'{col}_len', f'match_{col}_len']].max(axis = 1)
            
            df[f'{col}_nlcsk'] = df[f'{col}_lcs'] / df[f'match_{col}_len']
            df[f'{col}_nlcs'] = df[f'{col}_lcs'] / df[f'{col}_len']
            
            df = df.drop(f'{col}_len', axis = 1)
            df = df.drop(f'match_{col}_len', axis = 1)
            gc.collect()
            
    return df

## alphabet convert

In [22]:
def convert_japanese_alphabet(df: pd.DataFrame):
    kakasi = pykakasi.kakasi()
    kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
    kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
    kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
    conversion = kakasi.getConverter()

    def convert(row):
        for column in ["name", "address", "city", "state"]:
            try:
                row[column] = conversion.do(row[column])
            except:
                pass
        return row

    df[df["country"] == "JP"] = df[df["country"] == "JP"].progress_apply(convert, axis=1)
    return df

## for CV

In [23]:
# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
  return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
  return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

# KNNで選ばれたデータの正解のカバー率を計測する
def get_score(input_df: pd.DataFrame):
  scores = []
  for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
    targets = poi2ids[id2poi[id_str]]
    preds = set(matches.split())
    score = len((targets & preds)) / len((targets | preds))
    scores.append(score)
  scores = np.array(scores)
  return scores.mean()

In [54]:
from sklearn.model_selection import GroupKFold

## Dada process
if is_colab:
    data = pd.read_csv('../input/foursquare-location-matching/train.csv')
    tv_ids_d = np.load('/content/drive/MyDrive/datas/foursquare/traindata3/tv_ids_d.npy', allow_pickle=True).item()
    data = data.set_index('id')
    data = data.loc[tv_ids_d['valid_ids']]
    kf = GroupKFold(n_splits=20)
    for trn_idx, val_idx in kf.split(data, 
                                      data['point_of_interest'], 
                                      data['point_of_interest']):
      idx = val_idx
      break
    data = data.iloc[idx] 
    data = data.reset_index()
    # id --> poi変換用の辞書. １対１の関係
    id2poi = get_id2poi(data)
    # poi --> id変換用の辞書辞書. １対多の関係
    poi2ids = get_poi2ids(data)   
    data = data.drop('point_of_interest', axis = 1)

else :
    data = pd.read_csv('../input/foursquare-location-matching/test.csv')

    if len(data) < 20:
        data = pd.read_csv('../input/foursquare-location-matching/train.csv',
                          nrows = 100)
        data = data.drop('point_of_interest', axis = 1)
   
#need to new dataset
# data = convert_japanese_alphabet(data)

# special process
def get_lower(x):
    try:
        return x.lower()
    except:
        return x

for col in data.columns:
    if data[col].dtype == object and col != 'id':
        data[col] = data[col].apply(get_lower)
        
id2index_d = dict(zip(data['id'].values, data.index))

tfidf_d = {}
for col in vec_columns:
    tfidf = TfidfVectorizer()
    tv_fit = tfidf.fit_transform(data[col].fillna('nan'))
    tfidf_d[col] = tv_fit

out_df = pd.DataFrame()
out_df['id'] = data['id'].unique().tolist()
out_df['match_id'] = out_df['id']

test_data_simple = recall_simple(data)
test_data = recall_knn(data, NUM_NEIGHBOR, NUM_NEIGHBOR2)

print('train data by knn: %s' % len(test_data))
test_data = test_data.merge(test_data_simple,
                             on = ['id', 'match_id'],
                             how = 'outer')
del test_data_simple
gc.collect()

data = data.set_index('id')
print('Num of unique id: %s' % test_data['id'].nunique())
print('Num of test data: %s' % len(test_data))
print(test_data.sample(5))

  0%|          | 0/28471 [00:00<?, ?it/s]

Num of data: 23788
Num of data per id: 1.2805770887166237
Start knn grouped by country


  0%|          | 0/138 [00:00<?, ?it/s]

Start knn
train data by knn: 1145236
Num of unique id: 28471
Num of test data: 1147714
                      id          match_id     kdist  kneighbors  \
581624  E_6e38efe0275b98  E_73e377041fe897  1.699706        20.0   
320160  E_3edc343f3916e6  E_7bcc3dec026565  0.005804        11.0   
574786  E_3008f2461c1353  E_915978f4088b76  0.893472        20.0   
831284  E_324ae003e3a7d6  E_03ee54dd197fcb  0.039659        29.0   
554633  E_7aae30a870f76b  E_2780a1b7894d68  0.012414        19.0   

        kdist_country  kneighbors_country  
581624            NaN                 NaN  
320160            NaN                 NaN  
574786            NaN                 NaN  
831284            NaN                 NaN  
554633            NaN                 NaN  


In [55]:
# pair = pd.read_csv('../input/foursquare-location-matching/pairs.csv')
# test_data = pair[['id_1', 'id_2','match']]
# test_data.columns = ['id', 'match_id', 'match']
# test_data.head()

In [56]:
x = add_features(test_data[:10])
x['kdist_diff'] = (x['kdist'] - x['kdist_country']) /\
                            x['kdist_country']
x['kneighbors_mean'] = x[['kneighbors', 'kneighbors_country']].mean(axis = 1)

  0%|          | 0/10 [00:00<?, ?it/s]

In [58]:
for e in TRAIN_FEATURES:
    if e not in x : print(e)

In [59]:
## Model load
mtype = [
    'lightgbm',
    # 'lightgbm',
    # 'lightgbm',
#     'catboost','catboost','catboost'
]
models = [
    '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm3_fold0.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm3_fold1.txt',
    # '/content/drive/MyDrive/datas/foursquare/lightgbm/lgbm3_fold2.txt',
]
features = [
           TRAIN_FEATURES,
          #  TRAIN_FEATURES,
          #  TRAIN_FEATURES,
           ]

mix_model = []
for i, e in enumerate(mtype) :
    if e == 'catboost' :
      m = CatBoostClassifier()
      m.load_model(models[i])
      mix_model.append(m)
    else :
      mix_model.append(lgb.Booster(model_file = models[i]))

mix_model

In [ ]:
## Prediction
count = 0
start_row = 0
pred_df = pd.DataFrame()
unique_id = test_data['id'].unique().tolist()
num_split_id = len(unique_id) // NUM_SPLIT
for k in range(1, NUM_SPLIT + 1):
    print('Current split: %s' % k)
    end_row = start_row + num_split_id
    if k < NUM_SPLIT:
        cur_id = unique_id[start_row : end_row]
        cur_data = test_data[test_data['id'].isin(cur_id)]
    else:
        cur_id = unique_id[start_row: ]
        cur_data = test_data[test_data['id'].isin(cur_id)]
    
    # add features & model prediction
    cur_data = add_features(cur_data)
    cur_data['kdist_diff'] = (cur_data['kdist'] - cur_data['kdist_country']) /\
                                cur_data['kdist_country']
    cur_data['kneighbors_mean'] = cur_data[['kneighbors', 'kneighbors_country']].mean(axis = 1)

    preds = []
    for num, model in enumerate(mix_model) :
      if mtype[num] == 'catboost' :
          res = model.predict_proba(cur_data[features[num]])[:,1]
      else :
          res = model.predict(cur_data[features[num]])#,predict_disable_shape_check=True)
      cur_data['pred{}'.format(num)] = res
      preds.append('pred{}'.format(num))

#    cur_pred_df = cur_data[cur_data['pred'] > THRESHOLD][['id', 'match_id']]
    cur_pred_df = cur_data[['id','match_id']+preds]
    pred_df = pd.concat([pred_df, cur_pred_df])
    
    start_row = end_row
    count += len(cur_data)

    del cur_data, cur_pred_df
    gc.collect()
print(count)

Current split: 1


  0%|          | 0/10 [00:00<?, ?it/s]

Current split: 2


  0%|          | 0/10 [00:00<?, ?it/s]

Current split: 3


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
pred_df

In [ ]:
#pred_df['pred']=(pred_df['pred0'] + pred_df['pred1'] + pred_df['pred2'] + pred_df['pred3'] + pred_df['pred4'] + pred_df['pred5'])/6
# pred_df['pred']=(pred_df['pred0']+pred_df['pred1']+pred_df['pred2'])/3
pred_df['pred']=pred_df['pred0']

In [ ]:
pred_df

In [ ]:
pred_df = pred_df[pred_df['pred'] > THRESHOLD][['id', 'match_id']]
pred_df

In [ ]:
## Submission    
out_df = pd.concat([out_df, pred_df])
out_df = out_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
out_df = post_process(out_df)
print('Unique id: %s' % len(out_df))
print(out_df.head())

out_df[['id', 'matches']].to_csv('submission.csv', index = False)

In [ ]:
if is_colab:
  print(f"CV: {get_score(out_df):.6f}")

In [36]:
# 